In [3]:
from datasets import load_dataset

dataset = load_dataset("sst2")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset sst2 downloaded and prepared to /root/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
ls "results_a2t_word.8_sst2"

dpml.csv                 out_human.csv                  worst_reverted_text.csv
false_reverted_text.csv  reverted_text.csv
log.csv                  worst_false_reverted_text.csv


In [15]:
import os.path as osp
import pandas as pd

#dir_pth = './results_a2t_mlm_word.5_sst2'
#dir_pth = './results_a2t_mlm_word.8_sst2'
dir_pth = './results_a2t_word.8_sst2'
"""
out_df = pd.read_csv(osp.join(dir_pth, 'filtered_out.csv'))[
            ['original_text', 'perturbed_text', 'original_output',
             'perturbed_output', 'ground_truth_output',
            'result_type']]
"""
def load_dfs(dir_pth):
    out_df = pd.read_csv(osp.join(dir_pth, 'log.csv'))[
      ['original_text', 'perturbed_text', 'original_output',
       'perturbed_output', 'ground_truth_output',
       'result_type']]
       
    revert_df = pd.read_csv(osp.join(dir_pth, 'reverted_text.csv'))
    no_constraint_revert_df = pd.read_csv(osp.join(dir_pth, 'false_reverted_text.csv'))
    worst_revert_df = pd.read_csv(osp.join(dir_pth, 'worst_reverted_text.csv'))
    worst_no_constraint_revert_df = pd.read_csv(osp.join(dir_pth, 'worst_false_reverted_text.csv'))

    return out_df, revert_df, no_constraint_revert_df

test_out_df, test_revert_df, test_no_constraint_revert_df = load_dfs(dir_pth)
train_out_df, train_revert_df, train_no_constraint_revert_df = load_dfs(dir_pth + '_train')

[Adversarial GLUE](https://adversarialglue.github.io/)

In [16]:
def df_to_dataset(df, label_col='label'):
    dataset = Dataset.from_pandas(df)
    return dataset.class_encode_column(label_col)

In [17]:
from datasets import Dataset

out_df = train_out_df
perturbed_df = out_df[['perturbed_text',  'ground_truth_output']]
original_df = out_df[['original_text',  'ground_truth_output']]

perturbed_df.rename(columns = {'perturbed_text':'text', 
                              'ground_truth_output':'label'}, inplace = True)
original_df.rename(columns = {'original_text':'text', 
                               'ground_truth_output':'label'}, inplace = True)

test_revert_df.rename(columns = {'reverted_text':'text'}, inplace = True)
train_revert_df.rename(columns = {'reverted_text':'text'}, inplace = True)

test_no_constraint_revert_df.rename(columns = {'reverted_text':'text'}, inplace = True)
train_no_constraint_revert_df.rename(columns = {'reverted_text':'text'}, inplace = True)

#negative_df = perturbed_df[out_df['human'] == 'Negative']
#positive_df = perturbed_df[(out_df['human'] == 'Positive') | (out_df['human'] == 'Neutral')]
#every_df = perturbed_df[(out_df['human'] == 'Positive') | (out_df['human'] == 'Neutral') | (out_df['human'] == 'Negative')].sample(frac=0.5, replace=False, random_state=41)


import json
glue_dict = json.load(open("./adv_glue/dev.json"))
glue_sst2 = pd.DataFrame(glue_dict['sst2'])[['sentence', 'label']]

glue_sst2.rename(columns = {'sentence':'text'}, inplace = True)

train_dataset = Dataset.from_pandas(train_no_constraint_revert_df)
eval_dataset = Dataset.from_pandas(glue_sst2)

train_dataset = train_dataset.class_encode_column('label')
eval_dataset = eval_dataset.class_encode_column('label')

Stringifying the column:   0%|          | 0/23 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/23 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

Stringifying the column:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

  0%|          | 0/23 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [20]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2")

In [28]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(per_device_train_batch_size=8, output_dir="./logs/", evaluation_strategy="epoch", logging_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [29]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [31]:
# Train the model
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 22970
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8616


RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 10.91 GiB total capacity; 9.37 GiB already allocated; 66.75 MiB free; 9.40 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# Compute the evaluation metrics
trainer.evaluate()

In [ ]:
dir_pth

In [106]:
model.save_pretrained('./models/filtered_a2t_word.8_sst2_train')

Configuration saved in ./models/filtered_a2t_word.8_sst2_train/config.json
Model weights saved in ./models/filtered_a2t_word.8_sst2_train/pytorch_model.bin


In [ ]:
#model.save_pretrained('./models/unfiltered_a2t_mlm_word.5_sst2_test')
#model.save_pretrained('./models/unfiltered_a2t_mlm_word.8_sst2')
#model.save_pretrained('./models/finetune_unfiltered_a2t_word.8_sst2')
#model.save_pretrained('./models/finetune_filtered_a2t_word.8_sst2')

Configuration saved in ./models/finetune_filtered_a2t_word.8_sst2/config.json
Model weights saved in ./models/finetune_filtered_a2t_word.8_sst2/pytorch_model.bin


## Test Original Test Accuracy

### load model

In [ ]:
#model=model.from_pretrained('./models/finetune_filtered_a2t_word.8_sst2')
#model=model.from_pretrained('./models/finetune_unfiltered_a2t_word.8_sst2')
#model=model.from_pretrained('./models/unfiltered_a2t_mlm_word.8_sst2')
#model=model.from_pretrained('./models/unfiltered_a2t_mlm_word.5_sst2_test')

loading configuration file ./models/unfiltered_a2t_mlm_word.8_sst2/config.json
Model config BertConfig {
  "_name_or_path": "textattack/bert-base-uncased-SST-2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "finetuning_task": "sst-2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./models/unfiltered_a2t_mlm_word.8_sst2/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassi

In [107]:
from datasets import load_dataset

dataset = load_dataset("sst2")
train_dataset = dataset['train']
valid_dataset = dataset['validation']
test_dataset = dataset['test']

  0%|          | 0/3 [00:00<?, ?it/s]

### train eval

In [ ]:
# Run inferences with your new model using Pipeline
from transformers import pipeline

model.eval()
sentiment_model = pipeline("sentiment-analysis", model=model, 
                           tokenizer=tokenizer, device=0)

pred = sentiment_model(train_dataset['sentence'])
pred_labels = [0 if x['label'] == 'LABEL_0' else 1 for x in pred]

metric = load_metric("accuracy")
metric.compute(predictions=pred_labels, references=train_dataset['label'])

### validation eval

In [ ]:
model.eval()
sentiment_model = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0)

pred = sentiment_model(valid_dataset['sentence'])
pred_labels = [0 if x['label'] == 'LABEL_0' else 1 for x in pred]

metric = load_metric("accuracy")
metric.compute(predictions=pred_labels, references=valid_dataset['label'])

### test eval

In [ ]:
# Run inferences with your new model using Pipeline
from transformers import pipeline

model.eval()
sentiment_model = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0)

pred = sentiment_model(test_dataset['sentence'])
pred_labels = [-1 if x['label'] == 'LABEL_0' else 1 for x in pred]

metric = load_metric("accuracy")
metric.compute(predictions=pred_labels, references=test_dataset['label'])

## Original baseline

In [ ]:
original_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2")

loading configuration file https://huggingface.co/textattack/bert-base-uncased-SST-2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/293ab95645c102b941dee443ccf73fb9b5b5a9706b9893f09b5f1941b1bd0c8b.32da30c4245b376f0c4fd55aaf1c536c5ef13f10c248390e0311fcb4ca48f475
Model config BertConfig {
  "_name_or_path": "textattack/bert-base-uncased-SST-2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "finetuning_task": "sst-2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file 

In [ ]:
original_model.eval()
sentiment_model = pipeline("sentiment-analysis", model=original_model, 
                           tokenizer=tokenizer, device=0)

pred = sentiment_model(eval_dataset['text'])
pred_labels = [0 if x['label'] == 'LABEL_0' else 1 for x in pred]

metric.compute(predictions=pred_labels, references=eval_dataset['label'])

{'accuracy': 0.25}

In [ ]:
from datasets import load_dataset

dataset = load_dataset("sst2")

train_dataset = dataset['train']
valid_dataset = dataset['validation']
test_dataset = dataset['test']

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Run inferences with your new model using Pipeline
from transformers import pipeline

original_model.eval()
sentiment_model = pipeline("sentiment-analysis", model=original_model, 
                           tokenizer=tokenizer, device=0)

pred = sentiment_model(train_dataset['sentence'])
pred_labels = [0 if x['label'] == 'LABEL_0' else 1 for x in pred]

metric = load_metric("accuracy")
metric.compute(predictions=pred_labels, references=train_dataset['label'])

{'accuracy': 0.9864585962672051}

In [ ]:
original_model.eval()
sentiment_model = pipeline("sentiment-analysis", model=original_model, 
                           tokenizer=tokenizer, device=0)

pred = sentiment_model(valid_dataset['sentence'])
pred_labels = [0 if x['label'] == 'LABEL_0' else 1 for x in pred]

metric = load_metric("accuracy")
metric.compute(predictions=pred_labels, references=valid_dataset['label'])

{'accuracy': 0.9243119266055045}

In [ ]:
original_model.eval()
sentiment_model = pipeline("sentiment-analysis", model=original_model, 
                           tokenizer=tokenizer, device=0)

pred = sentiment_model(test_dataset['sentence'])
pred_labels = [-1 if x['label'] == 'LABEL_0' else 1 for x in pred]

metric = load_metric("accuracy")
metric.compute(predictions=pred_labels, references=test_dataset['label'])

{'accuracy': 0.4843492586490939}